In [73]:
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
from time import sleep
#import datetime as dt
from datetime import date
#from datetime import timedelta

In [69]:
date.today()

datetime.date(2024, 9, 8)

In [4]:
#Creación de la función que extrae el contenido de la web
def extractSoup(url):
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_soup = soup(webpage, "html.parser")
    return page_soup

def bot_send_text(bot_message):
    bot_token = "7517642163:AAH-QbrmfUYLljOzpDIbVKZFd8wxIN_ffUg"
    bot_chatID = "5544442" #Privado
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&text=' + bot_message
    response = requests.get(send_text)
    return response

def report(producto, categoria, link):
    mensaje = 'Se ha encontrado el producto ' + producto + ' que pertenece a la categoría ' + categoria + '. \n\n' + link
    bot_send_text(mensaje)


In [6]:
#Función que genera la lista con los productos e ids de la página
def scrapProductos(page_soup):
    for producto in page_soup.findAll('h3'):
        productos.append(producto.text[:-12])
        ids.append(producto.text[-10:])

#Función que genera la lista con las categorías de los productos
def scrapCategorias(page_soup):
    categoria = page_soup.findAll('li', {"class":"rt-categories"})
    for i in range(len(categoria)):
        categorias.append(categoria[i].get_text(strip=True))
    #return(categorias)

#Función que genera la lista con los links de los productos
def scrapLinks(page_soup):
    for i in page_soup.findAll('h3'):
        for j in i.find_all('a'):
            link = j.get('href')
            links.append(link)

#Función que genera la lista con las urls de las imágenes de los productos
def scrapImagenes(page_soup):
    for listing in page_soup.select('.listing-item .listing-thumb img'):
        img_url = listing.get('src')
        if img_url:
            images.append(img_url)

#Función que llama a todas las funciones de creación de listas
def scrapTodo(page_soup):
    scrapProductos(page_soup)
    scrapCategorias(page_soup)
    scrapLinks(page_soup)
    scrapImagenes(page_soup)

In [ ]:
productos = []
ids = []
categorias = []
links = []
images = []

In [71]:
# URL base de la página que vamos a analizar
base_url = "https://www.logrolimpio.com/catalogo-segunda-vida/?listing-page="
desfaseFecha = 2 #Desfase en días a reportar. 1 para ayer, 2 para antesdeayer, etc
fechaReporte = str(date.today()- timedelta(days=desfaseFecha))[2:].replace('-','')

In [7]:
pagina = 1
while pagina == 1 or page_soup.find('h3') != None:
    url = f"{base_url}{pagina}"
    page_soup = extractSoup(url)
    scrapTodo(page_soup)
    pagina +=1

In [8]:
df = pd.DataFrame({'id': ids,
                   'producto': productos,
                   'categoria': categorias,
                   'link': links,
                   'imagen': images},
                   columns=['id', 'producto', 'categoria', 'link', 'imagen'])

# Guardamos los datos en un archivo CSV
df.to_csv('productos.csv', index=False, encoding='utf-8-sig', sep=';')

In [78]:
for i in range(len(productos)):
    producto = productos[i]
    categoria = categorias[i]
    link = links[i]
    imagen = images[i]
    report(producto, categoria, link)
    sleep(20)

In [42]:
dfMuestra = df[['producto','categoria']][df['id'].str.startswith(fechaReporte)]

In [51]:
dfMuestra['producto'][0]

'Trona'

In [64]:
articulosDia = []
for i in range (len(dfMuestra)):
    articulosDia.append(dfMuestra['producto'][i] + '. Cat: ' + dfMuestra['categoria'][i])

In [65]:
articulosDia

['Trona. Cat: Accesorios bebés',
 'Patines talla 32-33. Cat: Material deportivo',
 'Lote de espejos. Cat: Elementos de decoración',
 'Cartera tejido neopreno. Cat: Mochilas, bolsos y maletas',
 'Juego de pesca. Cat: Juguetes',
 'Orinal. Cat: Accesorios bebés',
 'Frutero. Cat: Vajilla-Menaje',
 'Muñeco. Cat: Juguetes',
 'Estuche accesorios maquillaje. Cat: Otros Hogar',
 'Bolsa. Cat: Mochilas, bolsos y maletas',
 'Muñeco sujeta puertas. Cat: Otros Hogar',
 'Monitor 17″. Cat: Informática',
 'Pony. Cat: Juguetes',
 'Protectores para enchufe. Cat: Otros Niños Bebés',
 'Lote de cuadros. Cat: Elementos de decoración',
 'Botella de aluminio. Cat: Vajilla-Menaje',
 'Patines talla 28-30. Cat: Material deportivo']

In [77]:
report(producto, categoria, link)